In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

# Data loading

In [ ]:
_ = git.ensure_repo()

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

In [ ]:
# Process the deck data frame
deck_df = find_cards(deck_df, merge_data=True)

In [ ]:
deck_df

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_latest_data("deck")

if previous_df is not None:
    previous_df = previous_df.astype(deck_df[previous_df.columns.intersection(deck_df.columns)].dtypes.to_dict())

In [ ]:
if previous_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

In [ ]:
deck_df

In [ ]:
# Other

# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is not None:
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

In [ ]:
collection_df

In [ ]:
temp = deck_df.copy()
temp["Primary type"] = deck_df["Primary type"].fillna(deck_df["Card type"])
main_df = temp[temp["Section"] == "Main"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)
extra_df = temp[temp["Section"] == "Extra"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)
side_df = temp[temp["Section"] == "Side"].groupby(["Deck", "Primary type"])["Count"].sum().unstack(0)

In [ ]:
main_df

In [ ]:
extra_df

In [ ]:
side_df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import rgb_to_hsv, hex2color
from matplotlib.gridspec import GridSpec


def is_light_color(color):
    # Convert color to RGB if it's in hex format
    if isinstance(color, str):
        color = hex2color(color)
    # Convert RGB to HSV
    hsv = rgb_to_hsv(color)
    # Check the brightness (value in HSV)
    return hsv[2] > 0.6


def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct * total / 100.0))
        return f"{pct:.0f}%\n({val})"

    return my_autopct


cols = 3

colors_main = [plot.colors_dict[type] for type in main_df.index]
colors_extra = [plot.colors_dict[type] for type in extra_df.index]
remaining = side_df.index.difference(main_df.index.union(extra_df.index))
colors_main += [
    plot.colors_dict[type]
    for type in remaining
    if type not in ["Fusion Monster", "Synchro Monster", "Xyz Monster", "Link Monster"]
]
colors_main += [
    plot.colors_dict[type]
    for type in remaining
    if type in ["Fusion Monster", "Synchro Monster", "Xyz Monster", "Link Monster"]
]
colors_main = np.array(colors_main)
colors_extra = np.array(colors_extra)

ring_r = 0.3
decks = temp["Deck"].unique()

fig = plt.figure(figsize=(20, 6 * np.ceil(len(decks) / cols)))
fig.suptitle("Deck composition", fontsize=20)
gs = GridSpec(nrows=np.ceil(len(decks) / cols).astype(int), ncols=cols, hspace=0.2)

for i, deck in enumerate(decks):
    # Create sub-grid for pie and bar plots
    sub_gs = gs[(i // cols), i % cols].subgridspec(2, 1, height_ratios=[9, 1], hspace=0.2)

    # # Main plot in the upper sub-grid
    ax_pie = fig.add_subplot(sub_gs[0, 0])
    wedges1, texts1, autotexts1 = ax_pie.pie(
        main_df[deck].dropna(),
        autopct=make_autopct(main_df[deck].dropna()),
        startangle=90,
        radius=1,
        wedgeprops=dict(width=ring_r, edgecolor="w"),
        pctdistance=0.85,
        colors=colors_main[main_df[deck].notna()],
        counterclock=False,
        # frame=True,
    )

    if deck in extra_df.columns:
        wedges2, texts2, autotexts2 = ax_pie.pie(
            extra_df[deck].dropna(),
            autopct=make_autopct(extra_df[deck].dropna()),
            startangle=90,
            radius=1 - ring_r,
            wedgeprops=dict(width=ring_r, edgecolor="w"),
            pctdistance=0.75,
            colors=colors_extra[extra_df[deck].notna()],
            counterclock=False,
            # frame=True,
        )

    for wedge, text in zip(wedges1, autotexts1):
        color = wedge.get_facecolor()[:3]
        text.set_color("black" if is_light_color(color) else "white")
    for wedge, text in zip(wedges2, autotexts2):
        color = wedge.get_facecolor()[:3]  # Get the RGB color
        text.set_color("black" if is_light_color(color) else "white")

    ax_pie.text(0, 0, f"Main: {main_df[deck].sum()}\nExtra: {extra_df[deck].sum()}", ha="center", va="center", fontsize=14)
    ax_pie.set_title(deck, fontsize=16)
    ax_pie.set_xlim(-1, 1)
    ax_pie.set_ylim(-1, 1)
    ax_pie.set_aspect("equal", adjustable="box")

    ax_bar = fig.add_subplot(sub_gs[1, 0])  # Bar plot in the odd row
    ax_bar.axis("off")
    # Create bar plot in the lower sub-grid
    if deck in side_df and side_df[deck] is not None:
        sorted_side = side_df[deck].sort_values(ascending=True).dropna()
        side_total = sorted_side.sum()
        left = 0
        height = 0.1
        for j, (name, count) in enumerate(sorted_side.items()):
            left -= count
            color = plot.colors_dict[name]
            bc = ax_bar.barh(
                0,
                width=count,
                height=height,
                left=left,
                color=color,
                edgecolor="white",
            )
            ax_bar.bar_label(
                bc,
                labels=[f"{count/side_total*100:.0f}%\n({count})"],
                label_type="center",
                color="black" if is_light_color(color) else "white",
            )
        ax_bar.set_title(f"Side: {side_total}", fontsize=14)
        ax_bar.set_xlim(-side_total, 0)
        ax_bar.set_ylim(-0.05, 0.05)
        ax_bar.set_aspect(side_total, adjustable="box")

    else:
        ax_bar.set_title(f"Side: 0", fontsize=14)


# Create custom legend handles for main_df and extra_df
handles1 = [mpatches.Patch(color=plot.colors_dict[type], label=type) for type in main_df.index]
handles2 = [mpatches.Patch(color=plot.colors_dict[type], label=type) for type in extra_df.index]

# Adjust the legend position
fig.legend(
    handles=handles1,
    title="Main deck",
    loc="upper center",
    fontsize=12,
    ncol=len(handles1),
    bbox_to_anchor=(0.5, 0.97),
    frameon=False,
    title_fontsize=14,
)
fig.legend(
    handles=handles2,
    title="Extra deck",
    loc="upper center",
    fontsize=12,
    ncol=len(handles2),
    bbox_to_anchor=(0.5, 0.945),
    frameon=False,
    title_fontsize=14,
)
plt.subplots_adjust(top=0.90)

In [ ]:
ax = temp.plot.pie(subplots=True, autopct="%1.1f%%", figsize=(16, 4), title="Card type distribution")
for a in ax:
    a.set_ylabel(a.get_ylabel(), fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
temp = deck_df.groupby(["Deck", "Monster type"])["Count"].sum().unstack(0)
temp = temp[temp.notna().all(axis=1)]
ax = temp.plot.pie(subplots=True, autopct="%1.1f%%", figsize=(16, 4), title="Card type distribution")
for a in ax:
    a.set_ylabel(a.get_ylabel(), fontsize=16)

plt.tight_layout()
plt.show()